# NARU - Laboratory #

Dependencies:

- `conda install -c anaconda bcolz ` : Loading token / vector dictionary
- `conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch` : Math
- `conda install -c conda-forge spacy` : Tokenization

## Loading the dependencies ##

In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport utility.trainer
%aimport utility.embedding
%aimport utility.data_loader 
%aimport utility.ffnaru

Route loaded! Unit-Testing now...
Route Unit-Testing successful!
Source loaded! Unit-Testing now...
Source Unit-Testing successful!
Group loaded! Unit-Testing now...
Starting with:  torch.Size([1, 3])
i0f0t2d3 - t0 : h=torch.Size([1, 3]) r=torch.Size([1, 3])
i0f0t2d3 - t0 : h=torch.Size([1, 3]) r=torch.Size([1, 3])
Starting with:  torch.Size([1, 3])
i0f0t2d3 - t1 : h=torch.Size([1, 3]) r=torch.Size([1, 3])
i0f0t2d3 - t1 : h=torch.Size([1, 3]) r=torch.Size([1, 3])
i0f1t1d3 - t1 : s=torch.Size([1, 3])
i0f1t1d3 - t1 : h=torch.Size([1, 3]) r=torch.Size([1, 1])
[0.881648063659668, 0.9996621608734131]
Starting with:  torch.Size([1, 3])
i0f0t2d3 - t2 : h=torch.Size([1, 3]) r=torch.Size([1, 3])
i0f0t2d3 - t2 : h=torch.Size([1, 3]) r=torch.Size([1, 3])
i1f1t1d3 - t2 : s=torch.Size([1, 3])
i1f1t1d3 - t2 : h=torch.Size([1, 3]) r=torch.Size([1, 1])
i0f2t0d1 - t2 : s=torch.Size([1, 3])
i0f2t0d1 - t2 : s=torch.Size([1, 3])
tensor([[5.1944]])
tensor([[-0.0174,  0.1689, -2.5101],
        [-0.0058,  0.

In [ ]:
import torch
from utility.embedding import Encoder
from utility.ffnaru import Network
from utility.data_loader import load_jokes
from utility.trainer import exec_trial
from utility.persistence import save_params, load_params
import time

Loading Data : Jokes

In [ ]:

jokes = load_jokes()
training_data = jokes[:2]
test_data = jokes[2:4]

print(training_data[0])

## Training & Evaluation of Feed-Forward-NARU Model ##

In [7]:
# feed-forward-NARU
model = Network(
    depth=5,
    max_height=10,
    max_dim=64,
    max_cone=5,
    D_in=50,
    D_out=50
)

encoder = Encoder()

target_folder = 'models/feed-forward-NARU_'+time.strftime("%Y%m%d-%H%M%S")+'/'
model.set_params(load_params(target_folder))

for i in range(1): # Outer loop for saving the model from time to time...
    choice_matrices = exec_trial(
        model=model,
        encoder=encoder,
        optimizer=torch.optim.Adam(model.get_params(), lr=0.0001),
        training_data=training_data,
        test_data=test_data,
        epochs=1
    )
    print(choice_matrices)
    # SAVING PARAMETERS:
    save_params( model.get_params(), target_folder )


print('FFNN-NARU Training done!')

test_sentence = encoder.sequence_words_in('What did the bartender say to the jumper cables ?'.split())
preds = model.pred(test_sentence)
print(' '.join(encoder.sequence_vecs_in(preds)))



Stepping forward, current time: 0 ; Tokens: 9 ; Network depth: 5 ;
Starting with:  torch.Size([1, 50])
i0f0t8d50 - t0 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t0 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t0 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t0 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t0 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t0 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t0 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t0 : h=torch.Size([1, 50]) r=torch.Size([1, 275])

Stepping forward, current time: 1 ; Tokens: 9 ; Network depth: 5 ;
Starting with:  torch.Size([1, 50])
i0f0t8d50 - t1 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t1 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t1 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t1 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t1 : h=torch.Size([1, 50]) r=torch.S

i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t4 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i5f8t8d500 - t4 : s=torch

Starting with:  torch.Size([1, 50])
i0f0t8d50 - t7 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t7 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t7 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t7 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t7 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t7 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t7 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t7 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i2f1t15d275 - t7 : s=torch.Size([1, 50])
i2f1t15d275 - t7 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i2f1t15d275 - t7 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i2f1t15d275 - t7 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i2f1t15d275 - t7 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i2f1t15d275 - t7 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i2f1t15d275 - t7 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i2f1t15d275 - t7 : h=torch.Size([1, 2


Stepping forward, current time: 10 ; Tokens: 9 ; Network depth: 5 ;
Starting with:  torch.Size([1, 50])
i0f0t8d50 - t10 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t10 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t10 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t10 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t10 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t10 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t10 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t10 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i6f1t15d275 - t10 : s=torch.Size([1, 50])
i6f1t15d275 - t10 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t10 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t10 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t10 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t10 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t10 :


Stepping forward, current time: 13 ; Tokens: 9 ; Network depth: 5 ;
Starting with:  torch.Size([1, 50])
i0f0t8d50 - t13 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t13 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t13 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t13 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t13 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t13 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t13 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t13 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i6f1t15d275 - t13 : s=torch.Size([1, 50])
i6f1t15d275 - t13 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t13 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t13 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t13 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t13 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i6f1t15d275 - t13 :

...


Stepping forward, current time: 14 ; Tokens: 30 ; Network depth: 5 ;
Starting with:  torch.Size([1, 50])
i0f0t8d50 - t14 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t14 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t14 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t14 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t14 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t14 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t14 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t14 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i7f1t15d275 - t14 : s=torch.Size([1, 50])
i7f1t15d275 - t14 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i7f1t15d275 - t14 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i7f1t15d275 - t14 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i7f1t15d275 - t14 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i7f1t15d275 - t14 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i7f1t15d275 - t14 


Stepping forward, current time: 17 ; Tokens: 30 ; Network depth: 5 ;
Starting with:  torch.Size([1, 50])
i0f0t8d50 - t17 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t17 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t17 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t17 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t17 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t17 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t17 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t17 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i7f1t15d275 - t17 : s=torch.Size([1, 50])
i7f1t15d275 - t17 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i7f1t15d275 - t17 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i7f1t15d275 - t17 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i7f1t15d275 - t17 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i7f1t15d275 - t17 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i7f1t15d275 - t17 


Stepping forward, current time: 20 ; Tokens: 30 ; Network depth: 5 ;
Starting with:  torch.Size([1, 50])
i0f0t8d50 - t20 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t20 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t20 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t20 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t20 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t20 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t20 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t20 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i4f1t15d275 - t20 : s=torch.Size([1, 50])
i4f1t15d275 - t20 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i4f1t15d275 - t20 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i4f1t15d275 - t20 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i4f1t15d275 - t20 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i4f1t15d275 - t20 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i4f1t15d275 - t20 


Stepping forward, current time: 23 ; Tokens: 30 ; Network depth: 5 ;
Starting with:  torch.Size([1, 50])
i0f0t8d50 - t23 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t23 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t23 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t23 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t23 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t23 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t23 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f0t8d50 - t23 : h=torch.Size([1, 50]) r=torch.Size([1, 275])
i0f1t15d275 - t23 : s=torch.Size([1, 50])
i0f1t15d275 - t23 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i0f1t15d275 - t23 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i0f1t15d275 - t23 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i0f1t15d275 - t23 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i0f1t15d275 - t23 : h=torch.Size([1, 275]) r=torch.Size([1, 500])
i0f1t15d275 - t23 

KeyboardInterrupt: 

In [ ]:
matrix = [
    [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 4, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 4, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 4, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 4, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 4, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 4, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 4, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 4, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 3, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 9, 1, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 3, 5, 3, 0], [0, 3, 3, 0, 0], [0, 3, 5, 3, 0], [0, 3, 5, 0, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 4, 5, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 9, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 3, 1, 0], [0, 3, 8, 3, 0], [0, 3, 3, 1, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 3, 2, 0], [0, 3, 3, 2, 0], [0, 4, 3, 4, 0], [0, 4, 8, 4, 0], [0, 4, 8, 0, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 9, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 3, 1, 0], [0, 3, 8, 3, 0], [0, 3, 3, 1, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 3, 2, 0], [0, 3, 3, 2, 0], [0, 4, 3, 4, 0], [0, 4, 8, 4, 0], [0, 4, 8, 0, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 9, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 3, 1, 0], [0, 3, 8, 3, 0], [0, 3, 3, 1, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 3, 2, 0], [0, 3, 3, 2, 0], [0, 4, 3, 4, 0], [0, 4, 8, 4, 0], [0, 4, 8, 0, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 9, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 3, 1, 0], [0, 3, 8, 3, 0], [0, 3, 3, 1, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 3, 2, 0], [0, 3, 3, 2, 0], [0, 4, 3, 4, 0], [0, 4, 8, 4, 0], [0, 4, 8, 0, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 0, 0], [0, 4, 2, 0, 0], [0, 4, 8, 3, 0], [0, 1, 8, 1, 0], [0, 4, 1, 2, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0], [0, 0, 2, 2, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 9, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 3, 1, 0], [0, 3, 8, 3, 0], [0, 3, 3, 1, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 3, 2, 0], [0, 3, 3, 2, 0], [0, 4, 3, 4, 0], [0, 4, 8, 4, 0], [0, 4, 8, 0, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 4, 0, -1, -1], [0, 3, 4, 0, -1], [0, 3, 3, 4, 0], [0, 3, 3, 0, 0], [0, 4, 9, 4, 0], [0, 3, 8, 4, 0], [0, 3, 5, 0, 0], [0, 3, 3, 1, 0], [0, 4, 9, 4, 0], [0, 3, 8, 1, 0], [0, 3, 3, 1, 0], [0, 4, 3, 1, 0], [0, 3, 8, 3, 0], [0, 3, 3, 1, 0], [0, 3, 3, 4, 0], [0, 3, 3, 4, 0], [0, 3, 3, 2, 0], [0, 3, 3, 2, 0], [0, 4, 3, 4, 0], [0, 4, 8, 4, 0], [0, 4, 8, 0, 0]
    ], [
        [0, -1, -1, -1, -1], [0, 0, -1, -1, -1], [0, 2, 0, -1, -1], [0, 4, 2, 0, -1], [0, 4, 0, 2, 0], [0, 4, 8, 0, 0], [0, 4, 6, 3, 0], [0, 4, 0, 2, 0], [0, 2, 6, 2, 0], [0, 4, 2, 0, 0], [0, 4, 8, 2, 0], [0, 1, 8, 1, 0], [0, 4, 1, 3, 0], [0, 4, 8, 2, 0], [0, 0, 8, 2, 0], [0, 3, 6, 2, 0], [0, 3, 3, 1, 0], [0, 0, 3, 3, 0], [0, 0, 6, 3, 0]
    ]
]